In [1]:
import pandas as pd
import yaml
from sqlalchemy import create_engine

In [2]:
with open("../config/main.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

with open("../config/keys.yaml", "r") as f:
    keys = yaml.load(f, Loader=yaml.FullLoader)

import psycopg2

psy_engine = psycopg2.connect(
    database=config['database']['name'],
    user=config['database']['user'],
    password=keys['postgres'],
    host=config['database']['host'],
    port='5432'
)

cur = psy_engine.cursor()

In [3]:
# load in csv for attributes
df = pd.read_csv("/home/ubuntu/work/attributes/final.csv").iloc[:, 1:]

In [4]:
df = df.drop_duplicates(subset = 'corpusid')

In [5]:
sql = create_engine(f'postgresql://postgres:{keys["postgres"]}@{config["database"]["host"]}:5432/postgres')

In [20]:
df.to_sql('attributes', con=sql, if_exists='replace', chunksize = 10000)

ArgumentError: Could not parse rfc1738 URL from string '
    SELECT * FROM attributes;
'

In [18]:
from sqlalchemy import create_engine, text

engine = create_engine(f'postgresql://postgres:{keys["postgres"]}@{config["database"]["host"]}:5432/postgres')

# Query to pull in corpus IDs

sql = '''
    SELECT * FROM attributes;
'''

# Context manager to open the connection to the database and execute the query. 
with engine.connect() as conn:
    query = conn.execute(text(sql))
    
# Get all records and store in dataframe
attributes = pd.DataFrame(query.fetchall())

In [21]:
attributes.head()

,index,corpusid,externalids,url,title,authors,venue,publicationvenueid,year,referencecount,citationcount,influentialcitationcount,isopenaccess,s2fieldsofstudy,publicationtypes,publicationdate,journal,updated,id
0,0,52273203,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/39063bcd...,Large conformational changes of a highly dynam...,"{""{\""authorId\"": \""51285510\"", \""name\"": \""Isa...",Communications Biology,069e05e7-ca35-41d0-a8c7-bdc9ec6a82af,2018.0,28,13,0,True,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...",{JournalArticle},2018-09-03,"{""name"": ""Communications Biology"", ""pages"": nu...",2023-03-31T01:28:37.897Z,5692596
1,1,23708908,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/6ec7c156...,Drug sensitivity profiling identifies potentia...,"{""{\""authorId\"": \""5699001\"", \""name\"": \""H. K...",OncoTarget,106ecc86-1801-4b36-a05c-d36db69a968c,2017.0,46,24,0,True,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...",{JournalArticle},2017-10-31,"{""name"": ""Oncotarget"", ""pages"": ""97516 - 97527...",2023-04-05T06:18:15.837Z,5692609
2,2,13232625,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/c8566272...,Men's memory for women's sexual-interest and r...,"{""{\""authorId\"": \""4425040\"", \""name\"": \""T. T...",None,None,2011.0,36,13,0,True,"{""{\""category\"": \""Psychology\"", \""source\"": \...",{Review},2011-09-01,"{""name"": ""Applied Cognitive Psychology"", ""page...",2023-04-05T08:18:02.240Z,5692611
3,3,73484844,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/d4c9b2fa...,The Reactive Plasticity of Hippocampal Ionotro...,"{""{\""authorId\"": \""2225106\"", \""name\"": \""A. M...",International Journal of Molecular Sciences,8506a01a-40b8-4e6f-bbb8-ce2492139c15,2019.0,97,15,1,True,"{""{\""category\"": \""Biology\"", \""source\"": \""s2...","{Review,JournalArticle}",2019-02-27,"{""name"": ""International Journal of Molecular S...",2023-05-20T17:20:54.230Z,5692615
4,4,229159752,"{""ACL"": null, ""DBLP"": null, ""ArXiv"": null, ""MA...",https://www.semanticscholar.org/paper/7f13abe2...,High Mortality Rate in Adult COVID-19 Inpatien...,"{""{\""authorId\"": \""7685173\"", \""name\"": \""Saee...",Journal of Multidisciplinary Healthcare,12e5e550-28bd-401c-a348-f347956deec4,2020.0,37,17,1,True,"{""{\""category\"": \""Medicine\"", \""source\"": \""s...","{JournalArticle,Review}",2020-12-01,"{""name"": ""Journal of Multidisciplinary Healthc...",2023-05-23T13:34:07.581Z,5692620


In [30]:
psy_engine.commit()
cur.close()
psy_engine.close()